# Memory

In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
from langchain_core.chat_history import BaseChatMessageHistory

class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []

    def add_messages(self, messages):
        self.messages.extend(messages)

    def clear(self):
        self.messages = []

    def __repr__(self):
        return str(self.messages)

In [10]:
store = {}          # item(key=session_id, value=InMemoryHistory_인스턴스)

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [11]:
history_test = get_by_session_id('test')
history_test.add_messages(['hello', 'good morning', 'how are you?'])
history_test.add_messages(['I am fine', ' Thank you'])

history_test

['hello', 'good morning', 'how are you?', 'I am fine', ' Thank you']

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 {skill}을 잘하는 AI 어시스턴트야.'),
    MessagesPlaceholder(variable_name='history'),
    ('human', '{query}')    
])
model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)
chain = prompt | model


In [13]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'    
)

In [16]:
res = chain_with_history.invoke(
    {'skill': '대화', 'query': '다람쥐는 도토리 나무를 세 그루 키우고 있습니다.'},
    config={
        'configurable': {'session_id': 'squirrel'}
    }
)

print(res)

content='다람쥐가 도토리 나무를 세 그루 키우고 있다니, 정말 흥미로운 이야기네요! 도토리 나무는 다람쥐에게 중요한 식량 공급원이기도 하죠. 다람쥐는 도토리를 저장해 두고 겨울철에 먹기도 하고, 나무를 키우면서 자연 생태계에도 기여하게 됩니다. 이 다람쥐가 잘 자라고 있는지 궁금하네요! 어떤 이야기가 더 있을까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 46, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None} id='run-e934fda1-5591-420d-99cc-fa877183e873-0' usage_metadata={'input_tokens': 46, 'output_tokens': 112, 'total_tokens': 158, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [18]:
res = chain_with_history.invoke(
    {'skill': '대화', 'query': '토끼는 사과 나무를 다섯 그루 키우고 있습니다.'},
    config={
        'configurable': {'session_id': 'squirrel'}
    }
)

print(res)

content='토끼가 사과 나무를 다섯 그루 키우고 있다니 정말 귀엽고 재미있는 설정이에요! 사과 나무는 토끼에게 신선한 사과를 제공해 줄 수 있고, 나무 그늘에서 편안하게 쉴 수도 있겠네요. 토끼가 나무를 잘 돌보고 있는지, 또는 사과가 잘 자라고 있는지 궁금해지네요. 혹시 이 이야기에 추가할 내용이나 다른 동물들의 이야기도 있나요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 329, 'total_tokens': 439, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None} id='run-4f9018b0-d615-4180-8fc0-767b07cb3379-0' usage_metadata={'input_tokens': 329, 'output_tokens': 110, 'total_tokens': 439, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
